In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from scipy.signal import convolve2d as conv2

In [ ]:
def gaussian_pyramid(image):
    i0 = image.copy()
    gpi0 = [i0.astype(float)]
    i0 = np.array(i0)
    kernel_size = 13
    (x,y) = np.meshgrid(np.linspace(-1,1,kernel_size), np.linspace(-1,1,kernel_size))
    sigma = 1.0
    gaussian_blur = np.exp(-(x**2+y**2)/(2*sigma**2))
    gaussian_blur = gaussian_blur/np.sum(gaussian_blur)
    for i in range(4):
        #gaussian filter applied in each color channel
        i0r = conv2(i0[:,:,0], gaussian_blur, mode='same')
        i0g = conv2(i0[:,:,1], gaussian_blur, mode='same')
        i0b = conv2(i0[:,:,2], gaussian_blur, mode='same')
        i0 = np.stack((i0r,i0g,i0b), axis=-1)
        
        #down sampling: resizing
        w = int(i0.shape[1]*0.5)
        h = int(i0.shape[0]*0.5)
        dim = (w,h)
        i0 = cv.resize(i0, dim, interpolation = cv.INTER_AREA)
        print(i0.shape)
        #save to array for further processing
        gpi0.append(i0)
    
        plt.subplot(2,2,i+1)
        
        plt.imshow(i0/255)
        cv.imshow("scale", i0)
        cv.waitKey(0)
    
    cv.destroyAllWindows()
    return gpi0

In [ ]:
##teste 
img = cv.imread("/your/image")
img = cv.cvtColor(img,cv.COLOR_BGR2RGB)
gpyr = gaussian_pyramid(img)
for i in range(len(gpyr)):
    print(gpyr[i].shape)

In [ ]:
lapyr = [gpyr[3]]
i = 4
j = 0
##this certainly can be optimized
while(i>0):
    ##upsampling
    w = int(gpyr[i].shape[1]*2)
    h = int(gpyr[i].shape[0]*2)
    dim = (w,h)
    us = cv.resize(gpyr[i], dim, interpolation = cv.INTER_AREA)
    #us = cv.pyrUp(gpyr[i])
    
    ##check for approximation errors
    if(us.shape[0] < gpyr[i-1].shape[0]):
        x = np.zeros((1,us.shape[1],us.shape[2]))
        us = np.vstack((us,x))
    
    if(us.shape[1]<gpyr[i-1].shape[1]):
        x = np.zeros((us.shape[0],1,us.shape[2]))
        us = np.vstack((us,x))

    ##subtract finest and coarsest
    L = cv.subtract(gpyr[i-1], us)
    lapyr.append(L)
    
    i=i-1
    
    plt.subplot(2,2,j+1)
    plt.imshow(L)
    j= j+1
    cv.imshow("scale", L)
    cv.waitKey(0)
    
cv.destroyAllWindows()

In [ ]:
##test with cv2 given functions pyrUp and pyrDown
##gaussian pyr
GPimg = img.copy()
gpi0 = [GPimg]
for i in range(4):
    GPimg = cv.pyrDown(GPimg)
    gpi0.append(GPimg)
    plt.subplot(2,2,i+1)
        
    plt.imshow(GPimg)
    cv.imshow("scale", GPimg)
    cv.waitKey(0)
    
    cv.destroyAllWindows()
    
##laplacian pyr
lpimg = [gpi0[3]]
for i in range(3,0,-1):
    upscale = cv.pyrUp(gpi0[i])
    l = cv.subtract(gpi0[i-1], upscale)
    lpimg.append(l)
    plt.imshow(GPimg)
    cv.imshow("scale", GPimg)
    cv.waitKey(0)
    
    cv.destroyAllWindows()